In [1]:
import pandas as pd
import numpy as np

# Ścieżka do pliku
raw_file_path = '../data/accepted_2007_to_2018Q4.csv'

# Wybieram potrzebne kolumny
cols_to_keep = [
    # Zmienna celu (Target)
    'loan_status', 
    
    # Do Pricingu (Cena)
    'int_rate', 'installment', 'term',
    
    # Do Porównania z FICO (teza)
    'fico_range_low', 'fico_range_high', 'grade', 'sub_grade',
    
    # Zmienne Behawioralne i Finansowe (Model ML)
    'loan_amnt',          # Kwota
    'annual_inc',         # Dochód
    'dti',                # Dług do dochodu (Kluczowe!)
    'emp_length',         # Staż pracy
    'home_ownership',     # Mieszkanie
    'purpose',            # Cel kredytu
    'verification_status',# Weryfikacja
    'issue_d',            # Data udzielenia
    'revol_util',         # Wykorzystanie kart
    'revol_bal',          # Balans na kartach
    'pub_rec',            # Bankructwa publiczne
    'mort_acc',           # Liczba hipotek
    'earliest_cr_line'    # Początek historii kredytowej
]

# (bez nrows),tylko wybrane kolumny
df = pd.read_csv(raw_file_path, usecols=cols_to_keep, low_memory=False)

# Wyrzucam 'Current' (w trakcie spłaty), bo nie wiem czy splacą.
target_list = ['Fully Paid', 'Charged Off', 'Default']
df_clean = df[df['loan_status'].isin(target_list)].copy()

# Zamieniamy na liczby 
# 0 = Spłacił 
# 1 = Nie spłacił 
mapping_dict = {'Fully Paid': 0, 'Charged Off': 1, 'Default': 1}
df_clean['target'] = df_clean['loan_status'].map(mapping_dict)

# FICO mam Low i High. Uśredniam to do jednej liczby 'fico_score'.
df_clean['fico_score'] = (df_clean['fico_range_low'] + df_clean['fico_range_high']) / 2

# Zapisuje oczyszczony plik.
output_path = '../data/lending_club_cleaned.csv'

df_clean.to_csv(output_path, index=False)

print(df_clean['target'].value_counts(normalize=True))
display(df_clean.head())

target
0    0.80035
1    0.19965
Name: proportion, dtype: float64


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,...,dti,earliest_cr_line,fico_range_low,fico_range_high,pub_rec,revol_bal,revol_util,mort_acc,target,fico_score
0,3600.0,36 months,13.99,123.03,C,C4,10+ years,MORTGAGE,55000.0,Not Verified,...,5.91,Aug-2003,675.0,679.0,0.0,2765.0,29.7,1.0,0,677.0
1,24700.0,36 months,11.99,820.28,C,C1,10+ years,MORTGAGE,65000.0,Not Verified,...,16.06,Dec-1999,715.0,719.0,0.0,21470.0,19.2,4.0,0,717.0
2,20000.0,60 months,10.78,432.66,B,B4,10+ years,MORTGAGE,63000.0,Not Verified,...,10.78,Aug-2000,695.0,699.0,0.0,7869.0,56.2,5.0,0,697.0
4,10400.0,60 months,22.45,289.91,F,F1,3 years,MORTGAGE,104433.0,Source Verified,...,25.37,Jun-1998,695.0,699.0,0.0,21929.0,64.5,6.0,0,697.0
5,11950.0,36 months,13.44,405.18,C,C3,4 years,RENT,34000.0,Source Verified,...,10.20,Oct-1987,690.0,694.0,0.0,8822.0,68.4,0.0,0,692.0


**Charakterystyka rynku Peer-to-Peer (P2P) Lending**

Analizowane w pracy dane pochodzą z platformy Lending Club, działającej w modelu Peer-to-Peer (pożyczki społecznościowe). Model ten polega na bezpośrednim łączeniu kapitałodawców (inwestorów indywidualnych lub instytucjonalnych) z pożyczkobiorcami za pośrednictwem platformy internetowej, z pominięciem tradycyjnych instytucji bankowych.

W przeciwieństwie do sektora bankowego, gdzie ryzyko kredytowe jest agregowane przez instytucję, w modelu P2P ryzyko niewypłacalności dłużnika jest przenoszone bezpośrednio na inwestora. Sprawia to, że precyzyjna ocena wiarygodności kredytowej (scoring) i transparentność danych stają się kluczowymi czynnikami sukcesu. Specyfika tego rynku przyciąga często klientów z segmentu near-prime (o podwyższonym profilu ryzyka), co tłumaczy wysoki odsetek kredytów zagrożonych (default rate) w badanej próbie.